In [ ]:
!nvidia-smi -L

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model 
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import LSTM , Dropout , Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import cv2
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import string

In [ ]:
#downloading the dataset and caption
!gdown 1BJgZPftUv0oHAvYqGhVgjE8gX_aAnUY4
!gdown 1C8UED1FyiVPorNhUuCD2XFqS3g6jXTSM

In [ ]:
#unzip the images
!unzip -q /content/images.zip
!rm -rf /content/images.zip

In [ ]:
#we will shuffle the photo-caption dataset
filename = open('/content/captions.txt','r').readlines()
random.shuffle(filename)
#after shuffling we will save the .txt file
open('/content/captions.txt','w').writelines(filename)

In [ ]:
#creating a dictionary to store the photo , caption ... key -> index
photo_caption = dict()
filename = open('/content/captions.txt','r').read()
file_split = filename.split('\n')
#taking first 1000 lines
for i ,filename in tqdm(enumerate(file_split[:1000])):
  photo_caption_split = filename.split(',')
  photo_caption[i] = {"photo":photo_caption_split[0],
                  'caption':photo_caption_split[1]}

In [ ]:
photo_caption[0]

In [ ]:
#for feature extraction we will pretrain VGG16 model , you can use resnet , xception pretrain models too.
pretrained_models= tf.keras.applications.VGG16(include_top=True,weights="imagenet",pooling='avg')
output = Model(pretrained_models.input,pretrained_models.layers[-2].output) #model.layers[-2] will give fully connected layers

In [ ]:
output.summary()

In [ ]:
#single photo prediction
image = load_img(f'/content/image/Images/1000268201_693b08cb0e.jpg', target_size=(224, 224))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image) #normalize the image
feature = output.predict(image, verbose=0)
print(f'features : {feature[0]}')
print(f'feature len : {len(feature[0])}')

In [ ]:
#single caption preprocess
text = 'A person , riding a bicycle down a red ramp.'
print(f'raw text : {text}')
text = text.translate(str.maketrans('', '', string.punctuation))
print(f'punctuation removed text : {text}')
text = text.lower()
print(f'lower text : {text}')
text = text.split()
print(f'split text : {text}')
texts = [''.join(t) for t in text if t.isalpha() and len(t) >1]
print(f'final text after removing single char & non-alpha char : {texts}')
formatted_text = f"<start> {' '.join(texts)} <end>"
print(f'adding <start> <end> token text : {formatted_text}')

In [ ]:
#we will preprocess it for the 1000 images and caption
photo_cap_preprocess = dict()

for i , val in tqdm(photo_caption.items()):
  image = load_img(f'/content/image/Images/{val["photo"]}', target_size=(224, 224))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  feature = output.predict(image, verbose=0)


  text = val['caption']
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = text.lower()
  text = text.split()
  texts = [''.join(t) for t in text if t.isalpha() and len(t) >1]
  formatted_text = f"<start> {' '.join(texts)} <end>"
  

  photo_cap_preprocess[i]= {'photo_preprocess' :feature[0],
                        'text_preprocess' :formatted_text}

In [ ]:
#saving all text preprocess in a list for tokenizer
caption_tokenize = []
for i  , val in photo_cap_preprocess.items():
  caption_tokenize.append(val['text_preprocess'])

In [ ]:
tk = Tokenizer()

In [ ]:
tk.fit_on_texts(caption_tokenize)

In [ ]:
#creating  text to sequence
sentence = []
for caption in caption_tokenize:
    token_list = tk.texts_to_sequences([caption])[0]
    sentence.append(token_list)

In [ ]:
#how text to sequence work 
tk1 = Tokenizer()
ex = ['My name is souvik', 'I live in kolkata']
tk1.fit_on_texts(ex)
#vocab
print(f'vocab : {tk1.word_index}')

T2S = tk1.texts_to_sequences(ex)
print(f'actual text : {ex}')
print(f'text to sequence : {T2S}')

In [ ]:
max_len_sentence = max([len(i)for i in sentence])
print(f'max len sentence : {max_len_sentence}')

In [ ]:
total_words = len(tk.word_index)+1
print(f'VOCAB : {total_words}')

In [ ]:
#now the most intersting part sequence create , I have explained it in my blog , do check it out !
def sequence_create(max_len,total_words):
  the_sequence = [] 
  for j , vals in photo_cap_preprocess.items():
      token_list = tk.texts_to_sequences([vals['text_preprocess']])[0]
      for i in range(1, len(token_list)):
        input = token_list[:i]
        output = token_list[i]
        input = pad_sequences([input], maxlen=max_len-1)[0]
        output = to_categorical([output], num_classes=total_words)[0]
        the_sequence.append([photo_cap_preprocess[j]['photo_preprocess'],input,output])
  return the_sequence

In [ ]:
samples = sequence_create(max_len_sentence,total_words)

In [ ]:
samples[0][0] #photo feature

In [ ]:
samples[0][1] #caption preprocess

In [ ]:
samples[0][2] #output

In [ ]:
#creating a generator to process single batch at a time , explained it in my blog
def generator(samples, batch_size=256,shuffle_data=True):
    num_samples = len(samples)
    while True: 
        for offset in range(0, num_samples, batch_size):
          
            batch_samples = samples[offset:offset+batch_size]

            
            X_train = []
            y_train = []
            x_train = []

           
            for batch_sample in batch_samples:
                # print(batch_sample)
                img_name1 = batch_sample[0]
              
                x = batch_sample[1]
                y = batch_sample[2]
                
            
                X_train.append(img_name1)
                y_train.append(y)
                x_train.append(x)

            X_train = np.asarray(X_train)
      
            y_train = np.asarray(y_train)
            x_train = np.asarray(x_train)

            yield [X_train , x_train] , y_train

In [ ]:
#building the lstm model

def build_lstm():
  inputs1 = tf.keras.Input(shape=(4096,))
  fe2 = Dense(256, activation='relu')(inputs1)
  dp = Dropout(0.3)(fe2)

  input = Input(shape=(max_len_sentence-1,))
  mod = Embedding(total_words, 512 , mask_zero=True)(input)
  mod = LSTM(256,return_sequences=True )(mod)
  dp1 = Dropout(0.3)(mod)
  mod = LSTM(256)(dp1)

  added = layers.add([dp,mod])

  decoder2 = Dense(256, activation='relu')(added)

  outputs = Dense(total_words, activation='softmax')(decoder2)
  model = Model([inputs1,input],outputs)
  model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["Accuracy"])
  model.summary()
  return model

In [ ]:
model = build_lstm()

In [ ]:
train_data = generator(samples,batch_size=1024)

In [ ]:
x,y = next(train_data)

In [ ]:
print ('x_shape: ', x[0].shape)
print ('x_shape: ', x[1].shape)
print ('labels: ', y)

In [ ]:
modelcheck  = ModelCheckpoint('caption_best.h5',save_best_only=True,mode='min',monitor='loss')

In [ ]:
steps = len(samples)//1024
model.fit(train_data,epochs=100,callbacks=[modelcheck],steps_per_epoch=steps)

In [ ]:
best_model = load_model('/content/caption_best.h5')

In [ ]:
path = '/content/image/Images/3644142276_caed26029e.jpg'

im = cv2.imread(path)
image1 = load_img(path, target_size=(224, 224))
image = img_to_array(image1)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)
img2 = output.predict(image, verbose=0)
output_word = '<start>'
for i in range(max_len_sentence-1): 
  t_to_seq = tk.texts_to_sequences([output_word])[0]
  padding = pad_sequences([t_to_seq],maxlen=max_len_sentence-1)
  # print(padding)
  pred = np.argmax(best_model.predict([img2,padding]),axis=-1)
  # print(pred)
  for word, index in tk.word_index.items():
        if index == pred:
          if word == 'end':
            break
          output_word += ' ' + word
          
    
plt.imshow(im[:,:,::-1])
plt.axis('off')
plt.show()        

print(f'predicted caption : {output_word[7:]}.')


In [ ]:
model.reset_states()